In [1]:
import numpy as np
import pandas as pd
import orca
import os; os.chdir('../')
import warnings; warnings.simplefilter('ignore')

# Set data directory

d = '/home/data/fall_2018/'

if 'data_directory' in orca.list_injectables():
    d = orca.get_injectable('data_directory')
    

In [2]:
#from scripts import datasources, models, variables

In [3]:
from urbansim_templates import modelmanager

#modelmanager.initialize()

In [4]:
from pandas.compat import StringIO

@orca.table("TODs")
def TODs():
    data = StringIO("""TOD, lower_bound, upper_bound, TOD_id,
                       EA,        3.0,    6.0,        1
                       AM,        6.0,    9.0,        2
                       MD,        9.0,   15.5,        3
                       PM,       15.5,   18.5,        4
                       EV,       18.5,  999.0,        5
                       EV,        0.0 ,   3.0,        5
                    """)
    return pd.read_csv(data, sep=" *, *")

@orca.table("trips")
def trips():
    trips = pd.read_csv(d + '/HWtrips_101418.csv', dtype={'HHPER': 'S',
                                                        'HHPERTRIP': 'S'})

    #trips.head()

    TODs_df = orca.get_table("TODs").to_frame()

    # assign TOD to trips
    trips = (trips.assign(key=1)         
                .merge(TODs_df.assign(key=1), on="key")
                .query("lower_bound <= origin_ET < upper_bound")
                .drop(columns=['key', 'lower_bound', 'upper_bound'])
    )

    # make sure the process doesn't introduce duplicate trips
    assert max(trips.groupby("HHPERTRIP")
                   .size()
                   .reset_index(name="nrows")
                   .sort_values("nrows", ascending=False)
                   .loc[:, "nrows"]
           ) == 1

    #trips.head()
    return trips

In [5]:
trips_df = orca.get_table("trips").to_frame()

persons_TOD = (trips_df.groupby("HHPER")
                # select workers making 2 trips
          .filter(lambda x: len(x) == 2)
                # create a direction column with HW and WH
          .assign(direction = lambda x: np.where(x["origin"] == "home", "HW", "WH"))
          .rename(columns={"origin_ST": "trip_ST"})
          .pivot_table(index='HHPER', columns='direction', 
                       values=["origin_dwell", "origin_ET", "trip_ST", "TT", "MODE", "TOD", "TOD_id"], 
                       aggfunc='first')
          #.assign(TOD2 = lambda x: x.TOD_HW + '-' + x.TOD_WH)
          #.drop(columns=["home", "work"])
          #.reset_index()
          #.set_index("HHPER")
    )

persons_TOD.columns = persons_TOD.columns.map('_'.join).str.strip(" _")

# For MNL
persons_TOD = persons_TOD.assign(TOD2s = lambda x: x.TOD_HW + '-' + x.TOD_WH,
                                 TOD2i = lambda x: x.TOD_id_HW * 10 + x.TOD_id_WH)

persons_TOD.head()
               #.reset_index(name="nrows")
               #.sort_values("nrows", ascending=False)
               #.loc[:, "nrows"]


#trips.head()

#trips.pivot_table()

,MODE_HW,MODE_WH,TOD_HW,TOD_WH,TOD_id_HW,TOD_id_WH,TT_HW,TT_WH,origin_ET_HW,origin_ET_WH,origin_dwell_HW,origin_dwell_WH,trip_ST_HW,trip_ST_WH,TOD2s,TOD2i
HHPER,,,,,,,,,,,,,,,,
10352742,drive_alone,drive_alone,AM,PM,2,4,0.416667,2.416667,8.750000,16.75,13.583333,7.583333,19.166667,9.166667,AM-PM,24
10414944,drive_alone,drive_alone,AM,PM,2,4,0.250000,0.500000,8.500000,17.50,14.500000,8.750000,18.000000,8.750000,AM-PM,24
10417661,drive_alone,drive_alone,AM,PM,2,4,1.000000,1.250000,8.000000,17.50,13.250000,8.500000,18.750000,9.000000,AM-PM,24
10417662,drive_alone,drive_alone,AM,EV,2,5,0.500000,0.500000,8.000000,18.50,13.000000,10.000000,19.000000,8.500000,AM-EV,25
10436132,drive_alone,drive_alone,AM,EV,2,5,0.600000,0.400000,7.266667,19.00,11.866667,11.133333,19.400000,7.866667,AM-EV,25


## Prepare table for model selecting HW and WH departure times at once

In [6]:
@orca.table(cache=False)
def persons():
    #Use the CHTS persons processed with 
    # https://raw.githubusercontent.com/ual/ual_model_workspace/master/fall-2018-models/notebooks-max/WLCM_pre-processing.ipynb
    df = pd.read_csv(
        d + 'CHTS_csv_format/data/Deliv_PER.csv',
        #dtype={'SAMPN': 'S', 'PERNO': 'S'},
        #index_col = ["SAMPN", "PERNO"],
        usecols = ["SAMPN", "PERNO", "GEND", "RELAT", "AGE", "HISP",
                   "EMPLY", "WMODE", "EDUCA", "HOURS", "STUDE",
                   "RACE1"
                  ]
    )
    # rename columns to match those in complete persons table
    #person_id,member_id,age,primary_commute_mode,relate,edu,sex,
    #hours,hispanic,race_id,student,worker,household_id,
    #earning, work_at_home, node_id_small,node_id_walk,job_id
    
    # CHTS EDUCA
    # 1 NOT A HIGH SCHOOL GRADUATE, 12 GRADE OR LESS (THIS INCLUDES VERY
    # YOUNG CHILDREN TOO)
    # 2 HIGH SCHOOL GRADUATE (HIGH SCHOOL DIPLOMA OR GED)
    # 3 SOME COLLEGE CREDIT BUT NO DEGREE
    # 4 ASSOCIATE OR TECHNICAL SCHOOL DEGREE
    # 5 BACHELOR’S OR UNDERGRADUATE DEGREE
    # 6 GRADUATE DEGREE (INCLUDES PROFESSIONAL DEGREE LIKE MD, DDS, JD) 

    # CHTS RACE
    #01 White
    #02 Black or African American
    #03 American Indian or Alaska Native
    #04 Asian (Asian Indian, Japanese, Chinese, Korean, Filipino, Vietnamese)
    #05 Native Hawaiian or Pacific Islander (Guamanian, Samoan, Fijian)
    #97 Other (please specify) (O_RACE)
    #98 I do not know
    #99 I prefer not to answer 
    
    # CHTS WMODE
    # 01 WALK
    # 02 BIKE
    # 03 WHEELCHAIR / MOBILITY SCOOTER
    # 04 OTHER NON-MOTORIZED (Skateboard, etc.)
    # PRIVATE VEHICLE:
    # 05 AUTO / VAN / TRUCK DRIVER
    # 06 AUTO / VAN / TRUCK PASSENGER
    # 07 CARPOOL / VANPOOL
    # 08 MOTORCYCLE / SCOOTER / MOPED
    # PRIVATE TRANSIT:
    # 09 TAXI / HIRED CAR / LIMO
    # 10 RENTAL CAR / VEHICLE
    # 11 PRIVATE SHUTTLE (SuperShuttle, employer, hotel, etc.)
    # 12 GREYHOUND BUS
    # 13 AIRPLANE
    # 14 OTHER PRIVATE TRANSIT
    # PUBLIC TRANSIT:
    # BUS:
    # 15 LOCAL BUS / RAPID BUS
    # 16 EXPRESS BUS / COMMUTER BUS (AC Transbay, Golden Gate Transit, etc.)
    # 17 PREMIUM BUS (Metro Orange / Silver Line)
    # 18 SCHOOL BUS
    # 19 PUBLIC TRANSIT SHUTTLE (DASH, Emery Go-Round, etc.)
    # 20 AIRBART / LAX FLYAWAY
    # 21 DIAL-A-RIDE / ParaTransit (Access Services, etc.)
    # 22 AMTRAK BUS
    # 23 OTHER BUS RAIL/SUBWAY:
    # 24 BART, METRO RED / PURPLE LINE
    # 25 ACE, AMTRAK, CALTRAIN, COASTER, METROLINK
    # 26 METRO BLUE / GREEN / GOLD LINE, MUNI METRO, SACRAMENTO LIGHT
    # RAIL, SAN DIEGO SPRINTER / TROLLEY / ORANGE/BLUE/GREEN, VTA
    # LIGHT RAIL
    # 27 STREET CAR / CABLE CAR
    # 28 OTHER RAIL
    # FERRY:
    # 29 FERRY / BOAT 

    df = ( df.rename(index=str,
                     columns= {"SAMPN": "household_id", 
                                 "PERNO": "member_id",
                                 "GEND": "sex",
                                 "RELAT": "relate",
                                 "AGE": "age",
                                 "HISP": "hispanic", #TODO:coding
                                 #"EMPLY": "worker",
                                 "WMODE": "primary_commute_mode", #TODO:coding
                                 "EDUCA": "edu", #TODO: coding
                                 "HOURS": "hours",
                                 "RACE1": "race_id", #TODO: coding
                                })
             .assign(student = lambda x: np.where(x['STUDE'].isin([1, 2]), 1, 0), # full or part time student
                     sex = lambda x: np.where(x['sex'] == 9, np.NaN, x['sex']),   # handle refused
                     woker = lambda x: np.where(x['EMPLY'] == 1, 1, 0),
                    )
             .drop(columns=["EMPLY", "STUDE"])
         )
    return df

#persons_df = pd.read_csv(
#        d + '/CHTS_csv_format/data/Deliv_PER.csv',
#        dtype={'HHPER': 'S'},
#        index_col = ["SAMPN", "PERNO"]
#    )
persons_df = orca.get_table("persons").to_frame()

persons_df.columns


Index(['household_id', 'member_id', 'relate', 'sex', 'age', 'hispanic',
       'race_id', 'hours', 'primary_commute_mode', 'edu', 'student', 'woker'],
      dtype='object')

In [7]:
@orca.table(cache=False)
def households():
    #Use the CHTS persons processed with 
    # https://raw.githubusercontent.com/ual/ual_model_workspace/master/fall-2018-models/notebooks-max/WLCM_pre-processing.ipynb
    df = pd.read_csv(
        d + 'CHTS_csv_format/data/Deliv_HH.csv',
        #dtype={'SAMPN': 'S', 'PERNO': 'S'},
        #index_col = ["SAMPN", "PERNO"],
        usecols = ['SAMPN','HHVEH','HHBIC','OWN','INCOM','HHSIZ']
    )
    # rename columns to match those in complete households table
    # household_id,serialno,persons,building_type,cars,income,race_of_head,
    # hispanic_head,age_of_head,workers,state,county,tract,block_group,
    # children,tenure,recent_mover,block_group_id,single_family,unit_id,building_id
    
    df = ( df.rename(index=str,
                     columns= {"SAMPN": "household_id", 
                                 "HHVEH": "cars",
                                 "INCOM": "income", # category -> $
                                 "HHSIZ": "persons",
                                 #"OWN": "tenure", #TODO:coding
                                 "HHBIC": "bikes", #cannot be used as it is missing from complete households
                                })
             .assign(tenure = lambda x: np.where(x['OWN'] == 1, 1, 2),
                    )
             .drop(columns=["OWN"])
         )
    return df

#persons_df = pd.read_csv(
#        d + '/CHTS_csv_format/data/Deliv_PER.csv',
#        dtype={'HHPER': 'S'},
#        index_col = ["SAMPN", "PERNO"]
#    )
hh_df = orca.get_table("households").to_frame()

hh_df.columns

Index(['household_id', 'cars', 'bikes', 'income', 'persons', 'tenure'], dtype='object')

In [8]:
@orca.column("persons")
def female(persons):
    return persons.sex == 2

@orca.column("persons")
def HHPER(persons):
    return persons.household_id.map(str) + persons.member_id.map(str)

In [9]:
orca.broadcast(
    'households', 'persons', cast_index=True, onto_on='household_id')

In [10]:

pp_df = orca.get_table("persons").to_frame()

pp_tod = pd.merge(pp_df, persons_TOD.reset_index(), how="inner", on="HHPER")


#pp_tod.columns
#pp_tod["TOD2i"] = pd.Series(pp_tod.TOD2.values, dtype="category").cat.rename_categories(range(1, 22))
#pp_tod.TOD2i

orca.add_table('persons_tod', pp_tod)

@orca.column("persons_tod")
def HW_EA(persons):
    return persons.TOD2i.between(10, 19)

# TOD for HW and WH trips
TOD2s_df = pp_tod.groupby("TOD2i").size().reset_index(name="n")

TOD2s_df = (TOD2s_df.assign(HW_EA=lambda x: x["TOD2i"].between(10, 19)
                           )
                    .set_index("TOD2i")
            )

#TOD2s_df = TOD2s_df.set_index("TOD2i")

orca.add_table('TOD2s', TOD2s_df)

orca.broadcast(
    'TOD2s', 'persons_tod', cast_index=True, onto_on='TOD2i')

#TODs_df.index
#pp_tod.head()
TOD2s_df

,n,HW_EA
TOD2i,,
11,1,True
12,8,True
13,228,True
14,223,True
15,25,True
22,16,False
23,682,False
24,2787,False
25,482,False


In [11]:
from urbansim_templates import modelmanager
from urbansim_templates.models import SmallMultinomialLogitStep, \
     LargeMultinomialLogitStep, SegmentedLargeMultinomialLogitStep

from collections import OrderedDict

modelmanager.initialize()

TOD2s_df.index.values
m0 = SmallMultinomialLogitStep(
    tables=["persons_tod"],
    #tables=["persons_tod"],
    choice_column = "TOD2i",
    name = "TOD_choice_model_sm",
)
#m0.model_expression = ("HW_EA")  #not working
m0.model_expression = OrderedDict([
    ('intercept', [11, ]),
    ] )
    
m0.fit()

Registering model step 'auto_ownership'
Registering model step 'TOD_choice'
Registering model step 'primary_mode_choice'
Registering model step 'WLCM'
Log-likelihood at zero: 0.0000
Initial Log-likelihood: 0.0000
Estimation Time for Point Estimation: 0.10 seconds.
Final log-likelihood: 0.0000
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                     _chosen   No. Observations:                5,733
Model:             Multinomial Logit Model   Df Residuals:                    5,732
Method:                                MLE   Df Model:                            1
Date:                     Thu, 28 Mar 2019   Pseudo R-squ.:                     nan
Time:                             17:20:58   Pseudo R-bar-squ.:                 inf
AIC:                                 2.000   Log-Likelihood:                  0.000
BIC:                                 8.654   LL-Null:                         0.000
                   coef    std err

In [12]:
m1 = LargeMultinomialLogitStep(
          choosers="persons_tod",
          alternatives="TOD2s",
          choice_column="TOD2i"
    )

#m1.model_expression = ("HW_EA")      #working
m1.model_expression = OrderedDict([   #not working
    ('intercept', [21, ]),
    ] )

m1.fit()

KeyError: 'right_side'